In [54]:
# imports 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from torch.utils.data import random_split, DataLoader
from scipy import stats
from scipy.linalg import orthogonal_procrustes
from sklearn.decomposition import PCA
import csv

In [55]:
def compute_optimal_A_b_mu(X_np, r):
    mu = np.mean(X_np, axis=0)
    cov = (X_np - mu).T @ (X_np - mu)
    U, S, _ = np.linalg.svd(cov)
    Ur = U[:, :r]  # shape (d, r)
    
    A = Ur.T  # encoder projects from d->r: (r, d)
    b = np.zeros(r)  # no bias needed in latent space
    
    return A, b, mu

In [56]:
# python packaged affine autoencoder

class ClassicAffineAutoencoder(nn.Module):
    def __init__(self, input_dim, bottleneck_dim):
        super().__init__()
        self.encoder = nn.Linear(input_dim, bottleneck_dim, bias=True)
        self.decoder = nn.Linear(bottleneck_dim, input_dim, bias=True)

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [57]:
class OptimalAffineAutoencoder(nn.Module):
    def __init__(self, input_dim, r, A, b, mu):
        super().__init__()
        self.A = torch.tensor(A, dtype=torch.float32)
        self.b = torch.tensor(b, dtype=torch.float32)
        self.mu = torch.tensor(mu, dtype=torch.float32)

    def encoder(self, x):
        # Return centered projected latent space: Ur^T(x - mu)
        x_centered = x - self.mu  # center by mean
        UrT = self.A.T  # A = Ur Ur^T ⇒ A.T = Ur^T Ur ⇒ encoder ~ Ur^T
        return x_centered @ UrT  # [batch_size, d] x [d, r] → [batch_size, r]
    
    def decoder(self, z):
        # Reconstruct from latent space: Ur * z + mu
        return z @ self.A + self.mu  # [batch_size, r] x [r, d] → [batch_size, d]

    def forward(self, x):
        # Complete encode-decode cycle
        z = self.encoder(x)
        return self.decoder(z)

In [40]:
# NONLINEAR AUTOENCODER 
class NonlinearAutoencoder(nn.Module):
    def __init__(self, input_dim, bottleneck_dim, hidden_dim=5):
        super().__init__()
        
        # Encoder: input -> hidden -> hidden -> bottleneck
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, bottleneck_dim)
        )
        
        # Decoder: bottleneck -> hidden -> hidden -> output
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat
    

In [58]:
# this is a combination modeel that allows for training after instantiation of optimal theoretical weights. 
class OptimalTrainableAffineAutoencoder(nn.Module):
    def __init__(self, input_dim, r, A_init, b_init, mu_init):
        super().__init__()
        self.A = nn.Parameter(torch.tensor(A_init, dtype=torch.float32))  # (r, d)
        self.b = nn.Parameter(torch.tensor(b_init, dtype=torch.float32))  # (r,)
        self.mu = nn.Parameter(torch.tensor(mu_init, dtype=torch.float32))  # (d,)

    def encoder(self, x):
        x_centered = x - self.mu  # (batch, d)
        return x_centered @ self.A.T + self.b  # (batch, r)

    def decoder(self, z):
        return z @ self.A + self.mu  # (batch, d)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


In [59]:


# 3. Modify train_autoencoder to accept loaders instead of raw tensor
def train_autoencoder(model, train_loader, val_loader, num_epochs, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon = model(batch)
            loss = criterion(recon, batch)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item() * batch.size(0)

        avg_train_loss = total_train_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                recon = model(batch)
                loss = criterion(recon, batch)
                total_val_loss += loss.item() * batch.size(0)

        avg_val_loss = total_val_loss / len(val_loader.dataset)
        val_losses.append(avg_val_loss)


    return model, train_losses, val_losses









In [60]:
# validation function only code for the optimal affine autoencoder
def valOnlyOptimalAffineAutoencoder(model, val_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    model.eval()
    total_loss = 0
    criterion = nn.MSELoss()

    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            recon = model(batch)
            loss = criterion(recon, batch)
            total_loss += loss.item() * batch.size(0)

    avg_loss = total_loss / len(val_loader.dataset)
    return avg_loss 

In [61]:
# get data ready
X_df = pd.read_csv("assetReturns_ff3factor.csv")
X_np = X_df.to_numpy().astype(np.float32)
X_tensor = torch.tensor(X_np)

print(f"Data shape: {X_np.shape}")

# set dims and latent space size 
input_dim = X_np.shape[1]
r = 3  # bottleneck dimension

# compute the optimal params (only once, outside the loop)
A, b, mu = compute_optimal_A_b_mu(X_np, r)

# Initialize results storage
results = {
    'classic_mse': [],
    'optimal_mse': [],
    'trainable_mse': [],
    'nonlinear_mse': [],    
    'classic_train_loss': [],
    'classic_val_loss': [],
    'trainable_train_loss': [],
    'trainable_val_loss': [],
    'nonlinear_train_loss': [],
    'nonlinear_val_loss': [],
    'classic_factors': [],
    'optimal_factors': [],
    'trainable_factors': [],
    'nonlinear_factors': [],    
    'classic_train_histories': [],
    'trainable_train_histories': [],
    'classic_val_histories': [],
    'trainable_val_histories': [],
    'nonlinear_train_histories': [],
    'nonlinear_val_histories': []
}
  
    # split data into training and validation with current seed

n_samples = X_tensor.shape[0]
train_size = int(0.8 * n_samples)
val_size = n_samples - train_size

# Slice chronologically
train_data = X_tensor[:train_size]
train_indices = train_data.indices
val_data = X_tensor[train_size:]
val_indices = np.arange(train_size, len(X_np))
val_dates = X_df.index[val_indices]

# create the data loaders
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    
    # initialize models for this run
model_classic = ClassicAffineAutoencoder(input_dim, r)
model_optimal = OptimalAffineAutoencoder(input_dim, r, A, b, mu)
model_optimal_trainable = OptimalTrainableAffineAutoencoder(input_dim, r, A, b, mu)
model_nonlinear = NonlinearAutoencoder(input_dim, r, hidden_dim=128)
    
    # Train classic model
model_classic, losses_classic_train, losses_classic_val = train_autoencoder(
    model_classic, 
    train_loader,
    val_loader,
    70,
    0.001
    )
    
    # Validation only on optimal model
losses_optimal_val = valOnlyOptimalAffineAutoencoder(model_optimal, val_loader)
    
    # Train optimal trainable model
model_optimal_trainable, losses_trainable_train, losses_trainable_val = train_autoencoder(
    model_optimal_trainable,
    train_loader,
    val_loader,
    20,
    0.001
)

model_nonlinear, losses_nonlinear_train, losses_nonlinear_val = train_autoencoder(
    model_nonlinear,
    train_loader,
    val_loader,
    70,
    0.001
);
    
    # Store results for this run
results['classic_mse'].append(min(losses_classic_val))
results['optimal_mse'].append(losses_optimal_val)
results['trainable_mse'].append(min(losses_trainable_val))
results['nonlinear_mse'].append(min(losses_nonlinear_val))

results['classic_train_loss'].append(losses_classic_train[-1])
results['classic_val_loss'].append(losses_classic_val[-1])
results['trainable_train_loss'].append(losses_trainable_train[-1])
results['trainable_val_loss'].append(losses_trainable_val[-1])
results['nonlinear_train_loss'].append(losses_nonlinear_train[-1])
results['nonlinear_val_loss'].append(losses_nonlinear_val[-1])

    # Store full training histories
results['classic_train_histories'].append(losses_classic_train)
results['classic_val_histories'].append(losses_classic_val)
results['trainable_train_histories'].append(losses_trainable_train)
results['trainable_val_histories'].append(losses_trainable_val)
results['nonlinear_train_histories'].append(losses_nonlinear_train)
results['nonlinear_val_histories'].append(losses_nonlinear_val)

    # Convert to NumPy arrays
X_train_np = train_data.numpy()
X_val_np = val_data.numpy()

    # Load ground truth latent factors (full set) and slice val rows
F_true_full = pd.read_csv("latentFactors_ff3factor.csv").to_numpy().astype(np.float32)
F_true_tensor = torch.tensor(F_true_full[val_indices])

    # --------------------------------------------
    # PCA BASELINE: Fit on training data, evaluate on val
    # --------------------------------------------

    # Fit PCA on training data
pca = PCA(n_components=r)
pca.fit(X_train_np)

    # Reconstruct validation data
X_val_recon = pca.inverse_transform(pca.transform(X_val_np))

    # Compute MSE on validation set
pca_mse = np.mean((X_val_np - X_val_recon) ** 2)
print(f"PCA baseline MSE: {pca_mse:.6f}")
    
    # Transform validation data
Z_pca = pca.transform(X_val_np)

    # Align to true latent factors using Procrustes
F = F_true_tensor.numpy()
R_pca, _ = orthogonal_procrustes(Z_pca, F)
Z_pca_aligned = Z_pca @ R_pca

    # Compute correlation per factor
corr_pca = np.abs([
        np.corrcoef(Z_pca_aligned[:, i], F[:, i])[0, 1]
        for i in range(F.shape[1])
])
    
    # Compute reconstruction MSE on validation data
X_val_reconstructed = pca.inverse_transform(Z_pca)
mse_pca = np.mean((X_val_np - X_val_reconstructed) ** 2)

    # Store results
results.setdefault('pca_mse', []).append(mse_pca)
results.setdefault('pca_factors', []).append(corr_pca)

    # --------------------------------------------
    # FACTOR ANALYSIS: Procrustes-aligned correlations
    # --------------------------------------------

    # Get validation data as tensor
val_tensor = X_tensor[val_indices]

    # Helper function to align & compute correlation
def aligned_corr(model, X_val, F_true_val):
    with torch.no_grad():
        Z = model.encoder(X_val).cpu().numpy()
        F = F_true_val.cpu().numpy()
    R, _ = orthogonal_procrustes(Z, F)
    Z_aligned = Z @ R
    return np.abs([
        np.corrcoef(Z_aligned[:, i], F[:, i])[0, 1]
        for i in range(F.shape[1])
    ])

    # Compute and store factor correlations
results['classic_factors'].append(
    aligned_corr(model_classic, val_tensor, F_true_tensor)
)
results['optimal_factors'].append(
    aligned_corr(model_optimal, val_tensor, F_true_tensor)
)
results['trainable_factors'].append(
    aligned_corr(model_optimal_trainable, val_tensor, F_true_tensor)
)
results['nonlinear_factors'].append(
    aligned_corr(model_nonlinear, val_tensor, F_true_tensor)
)

FileNotFoundError: [Errno 2] No such file or directory: 'assetReturns_ff3factor.csv'

In [29]:
def print_results_summary(results):
    # MSE Results (Validation Loss)
    print("\n VALIDATION MSE RESULTS")
    
    classic_mse = np.array(results['classic_mse'])
    optimal_mse = np.array(results['optimal_mse'])
    trainable_mse = np.array(results['trainable_mse'])
    nonlinear_mse = np.array(results['nonlinear_mse'])  # NEW
    pca_mse = np.array(results['pca_mse'])  # NEW

    print(f"Classic Autoencoder:")
    print(f"  Mean MSE: {classic_mse.mean():.8f} ")
    print(f"  Median:   {np.median(classic_mse):.8f}")
    print(f"  Min/Max:  {classic_mse.min():.8f}  ")
    
    print(f"\nOptimal Autoencoder:")
    print(f"  Mean MSE: {optimal_mse.mean():.8f} ")
    print(f"  Median:   {np.median(optimal_mse):.8f}")
    print(f"  Min/Max:  {optimal_mse.min():.8f}  {optimal_mse.max():.8f}")
    
    print(f"\nTrainable Optimal Autoencoder:")
    print(f"  Mean MSE: {trainable_mse.mean():.8f} ")
    print(f"  Median:   {np.median(trainable_mse):.8f}")
    print(f"  Min/Max:  {trainable_mse.min():.8f}  {trainable_mse.max():.8f}")

    print(f"\nNonlinear Autoencoder:")
    print(f"  Mean MSE: {nonlinear_mse.mean():.8f} ")
    print(f"  Median:   {np.median(nonlinear_mse):.8f}")
    print(f"  Min/Max:  {nonlinear_mse.min():.8f}  {nonlinear_mse.max():.8f}")

    print(f"\nPCA Baseline:")
    print(f"  Mean MSE: {pca_mse.mean():.8f} ")
    print(f"  Median:   {np.median(pca_mse):.8f}")
    print(f"  Min/Max:  {pca_mse.min():.8f}  {pca_mse.max():.8f}")
    
    # Performance Comparison
    """""A
 
    print(f"\n PERFORMANCE COMPARISON")
    
    classic_vs_optimal = (classic_mse.mean() - optimal_mse.mean()) / classic_mse.mean() * 100
    classic_vs_trainable = (classic_mse.mean() - trainable_mse.mean()) / classic_mse.mean() * 100
    optimal_vs_trainable = (optimal_mse.mean() - trainable_mse.mean()) / optimal_mse.mean() * 100
    pca_vs_classic = (classic_mse.mean() - pca_mse.mean()) / classic_mse.mean() * 100
    pca_vs_optimal = (optimal_mse.mean() - pca_mse.mean()) / optimal_mse.mean() * 100
    pca_vs_trainable = (trainable_mse.mean() - pca_mse.mean()) / trainable_mse.mean() * 100
    
    print(f"Optimal vs Classic:     {classic_vs_optimal:+.2f}% improvement")
    print(f"Trainable vs Classic:   {classic_vs_trainable:+.2f}% improvement")
    print(f"Trainable vs Optimal:   {optimal_vs_trainable:+.2f}% improvement")
    print(f"PCA vs Classic:         {pca_vs_classic:+.2f}% improvement")
    print(f"PCA vs Optimal:         {pca_vs_optimal:+.2f}% improvement")
    print(f"PCA vs Trainable:       {pca_vs_trainable:+.2f}% improvement")
    """
    
    # Final Training/Validation Losses
    print(f"\n FINAL TRAINING/VALIDATION LOSSES")
    
    classic_train_final = np.array(results['classic_train_loss'])
    classic_val_final = np.array(results['classic_val_loss'])
    trainable_train_final = np.array(results['trainable_train_loss'])
    trainable_val_final = np.array(results['trainable_val_loss'])
    nonlinear_train_final = np.array(results['nonlinear_train_loss'])  # NEW
    nonlinear_val_final = np.array(results['nonlinear_val_loss'])  # NEW            
    
    print(f"Classic Autoencoder:")
    print(f"  Final Train Loss: {classic_train_final.mean():.8f} ")
    print(f"  Final Val Loss:   {classic_val_final.mean():.8f} ")
    print(f"  Train/Val Ratio:  {(classic_train_final.mean()/classic_val_final.mean()):.4f}")
    
    print(f"\nTrainable Optimal Autoencoder:")
    print(f"  Final Train Loss: {trainable_train_final.mean():.8f} ")
    print(f"  Final Val Loss:   {trainable_val_final.mean():.8f} ")
    print(f"  Train/Val Ratio:  {(trainable_train_final.mean()/trainable_val_final.mean()):.4f}")

    print(f"\nNonlinear Autoencoder:")
    print(f"  Final Train Loss: {nonlinear_train_final.mean():.8f} ")
    print(f"  Final Val Loss:   {nonlinear_val_final.mean():.8f} ")
    print(f"  Train/Val Ratio:  {(nonlinear_train_final.mean()/nonlinear_val_final.mean()):.4f}")       
    
    # Factor Analysis Results
    print(f"\n FACTOR RECOVERY ANALYSIS")

    classic_factors = np.array(results['classic_factors'])
    optimal_factors = np.array(results['optimal_factors'])
    trainable_factors = np.array(results['trainable_factors'])
    nonlinear_factors = np.array(results['nonlinear_factors'])  # NEW   
    pca_factors = np.array(results['pca_factors'])  # NEW
    
    print(f"Factor Correlations:")
    print(f"  Classic:           {classic_factors.mean():.4f}")
    print(f"  Optimal:           {optimal_factors.mean():.4f} ")
    print(f"  Trainable Optimal: {trainable_factors.mean():.4f} ")
    print(f"  Nonlinear:        {nonlinear_factors.mean():.4f} ")
    print(f"  PCA:               {pca_factors.mean():.4f} ")
    
    print(f"\nPer-Factor Correlations:")
    print(f"{'Factor':<8} {'Classic':<10} {'Optimal':<10} {'Trainable':<10} {'PCA':<10}")
    
    for i in range(classic_factors.shape[1]):
        factor_classic = classic_factors[:, i].mean()
        factor_optimal = optimal_factors[:, i].mean()
        factor_trainable = trainable_factors[:, i].mean()
        factor_pca = pca_factors[:, i].mean()
        factor_nonlinear = nonlinear_factors[:, i].mean()  # NEW
        print(f"{i+1:<8} {factor_classic:<10.4f} {factor_optimal:<10.4f} {factor_trainable:<10.4f} {factor_pca:<10.4f} {factor_nonlinear:<10.4f}")

    # Factor correlation tests
    classic_factors_flat = classic_factors.flatten()
    optimal_factors_flat = optimal_factors.flatten()
    trainable_factors_flat = trainable_factors.flatten()
    pca_factors_flat = pca_factors.flatten()
    nonlinear_factors_flat = nonlinear_factors.flatten()  # NEW

    # Summary Table
    print(f"\n SUMMARY TABLE")
    
    summary_df = pd.DataFrame({
        'Model': ['Classic', 'Optimal', 'Trainable', 'PCA', 'Nonlinear'],
        'MSE_Mean': [
            classic_mse.mean(), 
            optimal_mse.mean(), 
            trainable_mse.mean(), 
            pca_mse.mean(),
            nonlinear_mse.mean()  # NEW
        ],

        'Factor_Corr_Mean': [
            classic_factors.mean(), 
            optimal_factors.mean(), 
            trainable_factors.mean(),
            pca_factors.mean(),
            nonlinear_factors.mean()
        ]
    })
    
    print(summary_df.round(6))
print_results_summary(results)  

NameError: name 'results' is not defined

In [11]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from scipy.linalg import orthogonal_procrustes


def compare_latent_factors(model, X_tensor, F_true_path, val_indices=None):
    model.eval()
    with torch.no_grad():
        Z_learned = model.encoder(X_tensor).detach().cpu().numpy()
    Z_true_full = pd.read_csv(F_true_path).to_numpy()
    
    if val_indices is not None:
        Z_true = Z_true_full[val_indices, :]
    else:
        Z_true = Z_true_full
    
    min_samples = min(Z_learned.shape[0], Z_true.shape[0])
    min_factors = min(Z_learned.shape[1], Z_true.shape[1])
    
    Z_learned = Z_learned[:min_samples, :min_factors]
    Z_true = Z_true[:min_samples, :min_factors]
    
    Z_learned_centered = Z_learned - Z_learned.mean(axis=0)
    Z_true_centered = Z_true - Z_true.mean(axis=0)
    
    R, _ = orthogonal_procrustes(Z_learned_centered, Z_true_centered)
    Z_aligned = Z_learned_centered @ R
    
    print("Latent Factor Correlations (after Procrustes alignment):")
    correlations = []
    for i in range(Z_aligned.shape[1]):
        corr, _ = pearsonr(Z_aligned[:, i], Z_true_centered[:, i])
        correlations.append(corr)
        print(f"  Factor {i+1}: correlation = {corr:.4f}")
    
    return Z_aligned, Z_true_centered, correlations

def plot_training_curves(losses_train_baseline, losses_val_baseline, 
                         losses_val_optimal, losses_train_trainable, 
                         losses_val_trainable):
    plt.figure(figsize=(12, 5))
    
    # Plot 1: Training curves
    plt.subplot(1, 2, 1)
    plt.plot(losses_train_baseline, label='Classic AE - Train', linestyle='-', color='teal')
    plt.plot(losses_val_baseline, label='Classic AE - Val', linestyle='--', color='teal')
    plt.plot(losses_train_trainable, label='Trainable AE - Train', linestyle='-', color='magenta')
    plt.plot(losses_val_trainable, label='Trainable AE - Val', linestyle='--', color='magenta')
 
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)
    
    # Plot 2: Final comparison
    plt.subplot(1, 2, 2)
    models = ['Classic', 'Optimal', 'Trainable']
    train_final = [losses_train_baseline[-1], 0, losses_train_trainable[-1]]
    val_final = [losses_val_baseline[-1], losses_val_optimal, losses_val_trainable[-1]]
    
    x = np.arange(len(models))
    width = 0.35
    
    plt.bar(x - width/2, train_final, width, label='Train', alpha=0.8, color='pink')
    plt.bar(x + width/2, val_final, width, label='Validation', alpha=0.8, color='lightblue')
    plt.xlabel('Model Type')
    plt.ylabel('Final Loss')
    plt.title('Final Loss Comparison')
    plt.xticks(x, models)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def evaluate_models(model_baseline, model_optimal, model_trainable, X_tensor, A_opt, b_opt, 
                   F_true_path, losses_baseline_train, losses_baseline_val,
                   losses_optimal_val, losses_trainable_train, losses_trainable_val,
                   val_indices=None):

    # 2. Latent Factor Correlation
    print("LATENT FACTOR ANALYSIS:")
    print("\nClassic AE Factors:")
    try:
        _, _, corr_base = compare_latent_factors(model_baseline, X_tensor, F_true_path, val_indices)
        avg_corr_base = np.mean(np.abs(corr_base))
    except Exception as e:
        print(f"   Error in factor analysis: {e}")
        avg_corr_base = 0
    
    print(f"   Average |correlation|: {avg_corr_base:.4f}")
    
    print("\nOptimal AE Factors:")
    try:
        _, _, corr_opt = compare_latent_factors(model_optimal, X_tensor, F_true_path, val_indices)
        avg_corr_opt = np.mean(np.abs(corr_opt))
    except Exception as e:
        print(f"   Error in factor analysis: {e}")
        avg_corr_opt = 0
    
    print(f"   Average |correlation|: {avg_corr_opt:.4f}")

    print("\nTrainable AE Factors:")
    try:
        _, _, corr_train = compare_latent_factors(model_trainable, X_tensor, F_true_path, val_indices)
        avg_corr_train = np.mean(np.abs(corr_train))
    except Exception as e:
        print(f"   Error in factor analysis: {e}")
        avg_corr_train = 0

    print(f"   Average |correlation|: {avg_corr_train:.4f}")
  
    # 5. Plot training curves
    print("PLOTTING TRAINING CURVES...")
    plot_training_curves(losses_baseline_train, losses_val_baseline=losses_baseline_val,
                         losses_val_optimal=losses_optimal_val,
                         losses_train_trainable=losses_trainable_train,
                         losses_val_trainable=losses_trainable_val)


Theoretical autoencoder always outperforms on reconstruction MSE indicating a true optimal solution 
- theoretical has no training and classic has 300 epochs of training
- however i think that in order to best learn super noisy financial data, i think you need to train the optimal one and that will tailor it to your noisy dataset -> i gave the optimal solution ~50 epochs and its latent factor predictions were better than the classic autoencoder 
Question: I have tried training the optimal for about 20-50 epochs and it showed better performance than a classic that was trained for 300 epochs for nonlinear and GARCH data. Wondering if this is useful for CT data? should I try denoising in the latent space? because the latent factors are slightly worse than the classic but im guessing thats because we let the classic train and the optimal has no training at all. 